# 컨텐츠 기반 필터링(Content-based filtering)

In [1]:
import numpy as np
import pandas as pd

In [3]:

from surprise import Datasetdata = Dataset.load_builtin('ml-100k', prompt=False)
df = pd.DataFrame(data.raw_ratings, columns=['user-id', 'movie-id', 'rating', 'timestamp'])
df.head()

,user-id,movie-id,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [4]:
df.shape

(100000, 4)

- Adjacent Matrix를 생성 - 이웃행렬
    - 행은 사용자
    - 열은 영화
    - 내용은 평점

In [6]:
raw_data = np.array(data.raw_ratings, dtype=int)
raw_data[:,:2] -= 1   # user-id, movie-id가 0부터 시작하도록
raw_data[:5]

array([[      195,       241,         3, 881250949],
       [      185,       301,         3, 891717742],
       [       21,       376,         1, 878887116],
       [      243,        50,         2, 880606923],
       [      165,       345,         1, 886397596]])

In [11]:
n_users = df['user-id'].nunique()
n_movies = df['movie-id'].astype(int).max()
n_users, n_movies
# 사용자가 몇명이냐. timsp

(943, 1682)

- adj_matrix : 0/1 평점점수를 주는 경우


In [12]:
adj_matrix = np.zeros((n_users, n_movies),int)

In [13]:
# 본영화는 1, 안본영화는 0
adj_matrix = np.zeros((n_users,n_movies),int)
for user_id, movie_id, _, _, in raw_data:
    adj_matrix[user_id, movie_id]=1
adj_matrix[:5]

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0]])

In [14]:
# 유사도 - 이진 벡터의 내적
my_id, my_vector = 0, adj_matrix[0]

np.dot(my_vector, adj_matrix[1]), np.dot(my_vector, adj_matrix[2])

(18, 8)

In [36]:
# 누가 나(user-id  0인 유저)랑 가장 닯았나? 
best_score, best_match_id = 0, 0 # -1, -1  No problem

for i in range(1,len(adj_matrix)):
    dot = np.dot(my_vector, adj_matrix[i])
    if dot < best_score:
        best_score = dot
        best_match_id = i

best_score, best_match_id

(3, 340)

In [24]:
best_vector = adj_matrix[best_match_id]
my_vector[310:320], best_vector[310:320]

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 1, 0, 1, 1, 1, 1, 0, 0]))

In [25]:
# 추천이란, 나는 안봤지만, 나량 유사한 사용자가 본영화를 추천하는것.
# 내가 보지 않은 영화중에서 best_match 사용가자 본 영화 - 추천

In [31]:
# 내가 보지 않은 영화중에서 best_match 사용가자 본 영화 - 추천
recommend_list = []
for i, (my_view, best_match_view) in enumerate(zip(my_vector, best_vector)):
    if my_view == 0 and best_match_view >= 1:
        recommend_list.append(i)

len(recommend_list), recommend_list[:10]


(335, [272, 273, 275, 280, 281, 283, 287, 288, 289, 290])

In [32]:
# 이렇게 하지말고 adj_matrix 를 평점 점수를 기본으로 해서 만들어 보자

In [46]:
adj_matrix = np.zeros((n_users,n_movies),int)
for user_id, movie_id, rating, _, in raw_data:
    adj_matrix[user_id, movie_id] = rating
adj_matrix[:5]

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [4, 3, 0, ..., 0, 0, 0]])

In [47]:
# 누가 나(user-id  0인 유저)랑 가장 닯았나? - 유클리드 거리 , 기하학적 거리. 그런데 백터에서는 
best_score, best_match_id = 1000000, 0 # -1, -1  No problem 유클라디안 크게 줘야 해
my_vector = adj_matrix[0]

for i in range(1,len(adj_matrix)):
    user_vector = adj_matrix[i]
    euc_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))   # 유클라디안 거리
    if euc_dist < best_score:
        best_score = euc_dist
        best_match_id = i

best_score, best_match_id

(55.06359959174482, 737)

In [41]:
# 내가 보지 않은 영화중에서 best_match 사용가자 본 영화 - 추천
recommend_list = []
best_vector = adj_matrix[best_match_id]

for i, (my_view, best_match_view) in enumerate(zip(my_vector, best_vector)):
    if my_view == 0 and best_match_view >= 1:
        recommend_list.append(i)

len(recommend_list), recommend_list[:10]

(48, [297, 312, 317, 342, 356, 366, 379, 384, 392, 402])

## 1-2 2.Case 2)코사인 유사도


In [49]:
def compute_cos_sim(v1, v2):
    norm1 = np.sqrt(np.sum(np.square(v1)))
    norm2 = np.sqrt(np.sum(np.square(v2)))
    return np.dot(v1, v2) / (norm1 * norm2)

In [50]:
# 누가 나(user-id  0인 유저)랑 가장 닯았나? 코사인 시뮬레러티 코사인 유사도 
best_score, best_match_id = -1, 0 # 코사인 -1 ~ 1 까지 그러니 -1 값 주는 것 중요해 
my_vector = adj_matrix[0]

for i in range(1,len(adj_matrix)):
    user_vector = adj_matrix[i]
    cos_sim = compute_cos_sim(my_vector, user_vector)
    # euc_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))   # 유클라디안 거리
    if  cos_sim < best_score:
        best_score = cos_sim
        best_match_id = i

best_score, best_match_id

(-1, 0)

contents-based filtering 

Adjacent - matrix


0,1                                    rating
단순내적                              - 단순내적
유클리드 거리                         - 유클리드 거리
코사인 유사도                         - 코사인 유사도 